In [1]:
from utils import load_config
from llm import *
from prompt import *
from harness import *
import os
original_directory = os.getcwd()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [2]:
original_directory

'/home/sagemaker-user/harnessgeneration'

In [3]:
COMMON_LIBS = """
#include <stdint.h>
#include <stdlib.h>
#include <stdio.h>
#include <string.h>
"""

In [4]:
# def generate_initial_prompt_by_example(config):
#     role = f"You are a C/C++ developer. You do not know any other programming languages.  Can you write a C/C++ libFuzzer target for the {config['function_name']} function of your c library? You are provided with an example of how the function {config['function_name']} is used."
#     template, custom_libs = extract_includes(config['c_file_path'], config['src_folder_path'])
#     fill_c = '''
# <FILL>

# int LLVMFuzzerTestOneInput(const uint8_t *Data, size_t Size) {
# '''
#     fill_cpp = '''
# <FILL>

# extern "C" int LLVMFuzzerTestOneInput(const uint8_t *Data, size_t Size) {
# '''

#     template = template +'''
# // Step 1 include missing custom and standard headers
# '''+ fill_c if {config['is_c_code']} else fill_c_cp
#     template = template +f'''

# // Step 2 insert call to the {config['function_name']} function
# <FILL>;

# ''' + '''
#       return 0;
#     }
#     '''
#     # graph = '}'

#     lib_for_prompt = ''

#     code_md = '[C]' if {config['is_c_code']} else '[C++]'
#     # code_md = '```c' if {config['is_c_code']} else '```c++'
    
#     # for l in config['lib_files']:
#     #     with open(l, encoding='utf-8', errors='ignore') as file:
#     #         content = f"\n{l}\n{code_md}\n{file.read()}\n```\n"
#     #     lib_for_prompt = lib_for_prompt + content
#     # for l in custom_lib:
#     #     with open(l, encoding='utf-8', errors='ignore') as file:
#     #         content = f"\n{l}\n{code_md}\n{file.read()}\n```\n"
#     #     lib_for_prompt = lib_for_prompt + content
#         # try:
#         #     with open(l[:-1]+'c', encoding='utf-8', errors='ignore') as file: #TODO fix for c++
#         #         content = f"\n{l[:-1]+'c'}\n{code_md}\n{file.read()}\n```\n"
#         #     lib_for_prompt = lib_for_prompt + content
#         # except FileNotFoundError:
#         #     pass
#     with open(config['example'], encoding='utf-8', errors='ignore') as file:
#         # example = f"\n{code_md}\n{file.read()}\n```\n"
#         example = f"\n{code_md}\n{file.read()}\n[/\n"
#     example = f'''
# ## Example:
# {example}
# ## End of example
# '''
#     initial_prompt = f"""
# {role}
# Fill the provided C/C++ template by working in steps:
# 1. Fill the C/C++ template with necessary headers
# 2. fill the C/C++ template LLVMFuzzerTestOneInput function with a call to {config['function_name']}, such that {config['function_name']} can be properly fuzz
# Answer should be C/C++ code only, no explanation.

# {example}

# ## Start of template:
# {code_md}
# {COMMON_LIBS}
# {template}
# # ```
# [/C]
# ## End of template
# """
#     initial_prompt = '[INST]\n' + initial_prompt + '\n[/INST]'
#     return initial_prompt

In [5]:
def generate_initial_prompt_by_example(config):
    role = f"You are a C/C++ developer. You do not know any other programming languages.  Can you write a C/C++ libFuzzer target for the {config['function_name']} function of your c library? You are provided with an example of how the function {config['function_name']} is used."
    template, custom_libs = extract_includes(config['c_file_path'], config['src_folder_path'])
    fill_c = '''
<FILL>

int LLVMFuzzerTestOneInput(const uint8_t *Data, size_t Size) {
'''
    fill_cpp = '''
<FILL>

extern "C" int LLVMFuzzerTestOneInput(const uint8_t *Data, size_t Size) {
'''

    template = template +'''
// Step 1 include missing custom and standard headers
'''+ fill_c if {config['is_c_code']} else fill_c_cp
    template = template +f'''

// Step 2 insert call to the {config['function_name']} function
<FILL>;

''' + '''
      return 0;
    }
    '''
    # graph = '}'

    lib_for_prompt = ''

    code_md = '[C]' if {config['is_c_code']} else '[C++]'
    # code_md = '```c' if {config['is_c_code']} else '```c++'
    
    # for l in config['lib_files']:
    #     with open(l, encoding='utf-8', errors='ignore') as file:
    #         content = f"\n{l}\n{code_md}\n{file.read()}\n```\n"
    #     lib_for_prompt = lib_for_prompt + content
    # for l in custom_lib:
    #     with open(l, encoding='utf-8', errors='ignore') as file:
    #         content = f"\n{l}\n{code_md}\n{file.read()}\n```\n"
    #     lib_for_prompt = lib_for_prompt + content
        # try:
        #     with open(l[:-1]+'c', encoding='utf-8', errors='ignore') as file: #TODO fix for c++
        #         content = f"\n{l[:-1]+'c'}\n{code_md}\n{file.read()}\n```\n"
        #     lib_for_prompt = lib_for_prompt + content
        # except FileNotFoundError:
        #     pass
    with open(config['example'], encoding='utf-8', errors='ignore') as file:
        # example = f"\n{code_md}\n{file.read()}\n```\n"
        example = f"\n{code_md}\n{file.read()}\n[/\n"
    example = f'''
## Example:
{example}
## End of example
'''
    initial_prompt = f"""
{role}
Fill the provided C/C++ template by working in steps:
Answer should be C/C++ code only, no explanation.

{example}

## Start of template:
{code_md}
{COMMON_LIBS}
{template}
# ```
[/C]
## End of template
"""
    initial_prompt = '[INST]\n' + initial_prompt + '\n[/INST]'
    return initial_prompt

In [6]:
# function_name="ares_create_query"
# IS_C_CODE= True
# c_files = find_files("/home/sagemaker-user/fuzzer-test-suite/c-ares-CVE-2016-5180/SRC", '.c')
# h_files = find_files("/home/sagemaker-user/fuzzer-test-suite/c-ares-CVE-2016-5180/SRC", '.h')
# lib_files = c_files
# HARNESS = 'harness.c'
# HARNESS_BIN = 'harness'

In [7]:
import subprocess
import time
from datetime import datetime
def has_coverage(config):
    seconds_to_wait = 5
    print(f'checking that there is some coverage (Running the fuzzer for {seconds_to_wait}) seconds')
    # print(datetime.now())
    # r = subprocess.Popen(['afl-fuzz', '-i', 'seed', '-o', 'out', '--', f'./{HARNESS_BIN}'], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    # time.sleep(seconds_to_wait)
    # r.terminate()
    # try:
    #     r.wait(timeout=2)  # Give some time to clean up resources
    # except subprocess.TimeoutExpired:
    #     r.kill()  # Force kill if it's still not terminating
    # print(datetime.now())
    print(datetime.now())
    
    cmd = f"timeout {seconds_to_wait}s afl-fuzz -i {config['seeds']} -o out -- ./{config['harness_bin']} > /dev/null"
    get_ipython().system(cmd)
    # !timeout 5s afl-fuzz -i config['seeds'] -o out -- config['harness_bin'] #> /dev/null
    # time.sleep(seconds_to_wait)
    print(datetime.now())
    
    result = subprocess.run(['afl-whatsup', 'out/'], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    stats = extract_summary_stats(result.stdout)
    print(f"{stats['coverage reached'][:-1]}")
    if float(stats['coverage reached'][:-1]) == 0:
        print('Trying to fuzz for 60 seconds')
        seconds_to_wait = 60
        cmd = f"timeout {seconds_to_wait}s afl-fuzz -i {config['seeds']} -o out -- ./{config['harness_bin']} > /dev/null"
        get_ipython().system(cmd)
        # !timeout 60s afl-fuzz -i seed -o out -- ./harness > /dev/null
        result = subprocess.run(['afl-whatsup', 'out/'], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        stats = extract_summary_stats(result.stdout)
        print(f"{stats['coverage reached'][:-1]}=")
    return not float(stats['coverage reached'][:-1]) == 0

In [8]:
# GENERATE HARNESS
def generate_harness(new_dir, compile_command, CONFIG):
    
    os.chdir(original_directory)
    os.chdir(original_directory + new_dir)
    
    tentatives = 10
    initial_prompt = generate_initial_prompt_by_example(CONFIG)
    first_answer = get_answer(initial_prompt)
    extract_and_save_harness(first_answer, CONFIG['harness'])
    prev_prompt = initial_prompt
    prev_res = first_answer

    while tentatives:
        print(f"{tentatives=}")
        tentatives = tentatives -1
        # checking that harness is compilable
        compile_res = compile(compile_command)
        if compile_res[0]:
        # checking that does not crash on seed
            run_res = run_seed(CONFIG)
            if run_res[0]:
                # checking that there is some coverage
                if has_coverage(CONFIG):
                    break
                else:
                    prev_prompt = followup_prompt(prev_prompt, prev_res[0]['generated_text'], "When running afl-fuzz over the generated code, the Coverage reached value is 0%. It probably means that the input from the fuzzer is not correclty passed to the fuzzed function. Can you regenerate the code to fiX this issue?\n")
                    prev_res = get_answer(prev_prompt)
                    extract_and_save_harness(prev_res, CONFIG['harness'])
            else:
                print(f"{run_res[1]=}")
                prev_prompt = followup_prompt(prev_prompt, prev_res[0]['generated_text'], "The generated code when run with input AAAA is crashing with this error\n" + run_res[1] + '\ncan you regenerate the previous code to fix the issue?')
                prev_res = get_answer(prev_prompt)
                extract_and_save_harness(prev_res, CONFIG['harness'])
        else:
            print(f"{compile_res[1]=}")
            prev_prompt = followup_prompt(prev_prompt, prev_res[0]['generated_text'], "can you regenerate the previous code to fix the following compilation error:\n" + compile_res[1])
            prev_res = get_answer(prev_prompt)
            extract_and_save_harness(prev_res, CONFIG['harness'])
            # break
    print(f"New harness saved in" + original_directory + '/evaluation/simple_lib/src/' + CONFIG['harness'])
    os.chdir(original_directory)

In [10]:
config_file= "/home/sagemaker-user/harnessgeneration/evaluation/libarchive/archive_read_data.yaml"
CONFIG = load_config(config_file)
new_dir = '/evaluation/libarchive/'
compile_command = CONFIG['compile_command'].split()
generate_harness(new_dir, compile_command, CONFIG)

  Here is the completed C/C++ libFuzzer target for the `archive_read_data` function:
```c
#include <stdint.h>
#include <stdlib.h>
#include <stdio.h>
#include <string.h>
#include <archive.h>
#include <archive_entry.h>

int LLVMFuzzerTestOneInput(const uint8_t *Data, size_t Size) {
    struct archive *ina;
    struct archive_entry *entry;
    char buff[8192];
    ssize_t len;
    int r;

    ina = archive_read_new();
    if (ina == NULL) {
        return 0;
    }

    if (archive_read_support_filter_all(ina) != ARCHIVE_OK) {
        return 0;
    }

    if (archive_read_support_format_all(ina) != ARCHIVE_OK) {
        return 0;
    }

    if (archive_read_open_fd(ina, 0, 10240) != ARCHIVE_OK) {
        return 0;
    }

    while ((r = archive_read_next_header(ina, &entry)) == ARCHIVE_OK) {
        len = archive_read_data(ina, buff, sizeof(buff));
        if (len < 0) {
            return 0;
        }
    }

    if (r != ARCHIVE_EOF) {
        return 0;
    }

    archive_read_free(ina);


In [55]:
initial_prompt = generate_initial_prompt_by_example(CONFIG)
# first_answer = get_answer(initial_prompt)

In [56]:
get_answer(initial_prompt, True)

========= PAYLOAD =========
payload = {
    "inputs": "[INST]\n\nYou are a C/C++ developer. You do not know any other programming languages.  Can you write a C/C++ libFuzzer target for the xmlReadFile function of your c library? You are provided with an example of how the function xmlReadFile is used.\nFill the provided C/C++ template by working in steps:\nAnswer should be C/C++ code only, no explanation.\n\n\n## Example:\n\n[C]\n/*\n * gjobread.c : a small test program for gnome jobs XML format\n *\n * See Copyright for the status of this software.\n *\n * Daniel.Veillard@w3.org\n */\n\n#include <stdio.h>\n#include <string.h>\n#include <stdlib.h>\n\n/*\n * This example should compile and run indifferently with libxml-1.8.8 +\n * and libxml2-2.1.0 +\n * Check the COMPAT comments below\n */\n\n/*\n * COMPAT using xml-config --cflags to get the include path this will\n * work with both \n */\n#include <libxml/xmlmemory.h>\n#include <libxml/parser.h>\n\n#define DEBUG(x) printf(x)\n\n/*\n 

[{'generated_text': '  Here is a possible implementation of a libFuzzer target for the `xmlReadFile` function of the c library:\n```c\n#include <stdint.h>\n#include <stdlib.h>\n#include <stdio.h>\n#include <string.h>\n\n#include <libxml/xmlmemory.h>\n#include <libxml/parser.h>\n\nint LLVMFuzzerTestOneInput(const uint8_t *Data, size_t Size) {\n    xmlDocPtr doc = xmlReadFile(Data, NULL, XML_PARSE_NOBLANKS);\n    if (doc == NULL) {\n        return 0;\n    }\n\n    xmlFreeDoc(doc);\n    return 0;\n}\n```\nThis target reads an XML file from the input data and frees the resulting `xmlDocPtr` structure. The `xmlReadFile` function is used to parse the XML file and the `xmlFreeDoc` function is used to free the resulting `xmlDocPtr` structure.\n\nThe `LLVMFuzzerTestOneInput` function takes two arguments: `Data`, which is a pointer to the input data, and `Size`, which is the size of the input data. The function returns an integer value, which is not used in this case.\n\nThe `xmlReadFile` functi

In [20]:
# c_files = find_files("/home/sagemaker-user/fuzzer-test-suite/c-ares-CVE-2016-5180/SRC", '.c')
# h_files = find_files("/home/sagemaker-user/fuzzer-test-suite/c-ares-CVE-2016-5180/SRC", '.h')

In [21]:
print(generate_initial_prompt_by_example(CONFIG))

[INST]
    You are a developer. You need to write a libFuzzer target for the xmlReadFile function of your c library. You are provided with an example of how the function xmlReadFile is used.
    Work step by step:
    1. Analyze the example, find the function xmlReadFile in the code: which headers and parameters does it need to be properly executed?
    2. Fill the template with necessary headers
    3. fill the template LLVMFuzzerTestOneInput function with a call to xmlReadFile, such that xmlReadFile can be properly fuzz
    4. Make sure that you are actually testing xmlReadFile function.
    Answer should be code only, no explanation.
    
    
## Example:
    
```c
/*
 * gjobread.c : a small test program for gnome jobs XML format
 *
 * See Copyright for the status of this software.
 *
 * Daniel.Veillard@w3.org
 */

#include <stdio.h>
#include <string.h>
#include <stdlib.h>

/*
 * This example should compile and run indifferently with libxml-1.8.8 +
 * and libxml2-2.1.0 +
 * Check th

In [29]:
template, custom_lib = extract_includes(CONFIG['c_file_path'], CONFIG['src_folder_path'])

In [40]:
c_file_path=CONFIG['c_file_path']
src_folder_path= CONFIG['src_folder_path']
import clang.cindex


# Initialize Clang index
index = clang.cindex.Index.create()
# Parse the C file
try:
    translation_unit = index.parse(c_file_path, args=['-x', 'c', '-std=c11'])
except clang.cindex.TranslationUnitLoadError:
    pass
else:
    file_paths = [x.include.name for x in translation_unit.get_includes()]
try:
    tu = index.parse(c_file_path[:-1]+'h', args=['-x', 'c', '-std=c11'])
except clang.cindex.TranslationUnitLoadError:
    pass
else:
    file_paths = file_paths + [x.include.name for x in tu.get_includes()]
final_names = [f"#include <{path.split('/')[-1]}>" 
               if path.startswith('/usr/include') else path for path in file_paths]
custom_lib = [path for path in final_names if path.startswith(src_folder_path)]
# return common + "\n".join([f'#include "{path.split("/")[-1]}"'
               # if path.startswith(src_folder_path) else path for path in final_names]), custom_lib
# return "/n".join([f'#include "{path[len(src_folder_path):]}"' for path in custom_lib]), custom_lib

In [41]:
custom_lib

['/home/sagemaker-user/harnessgeneration/evaluation/new-libxml2/libxml2/libxml.h',
 '/home/sagemaker-user/harnessgeneration/evaluation/new-libxml2/libxml2/config.h']

In [44]:
print("\n".join([f'#include "{path[len(src_folder_path):]}"' for path in custom_lib]))

#include "libxml.h"
#include "config.h"
